<a href="https://colab.research.google.com/github/Gonzy/tests1/blob/master/ResumableLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics --upgrade -q > /dev/null

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive/MyDrive/datasets/

Mounted at /content/gdrive
project_1  test.zip


In [3]:
!mkdir -p /content/merged
!cp /content/gdrive/MyDrive/datasets/project_1/merged.tar.gz /content/merged.tar.gz

In [4]:
!tar -xzvf merged.tar.gz -C /content/merged > /dev/null

In [ ]:
#!cp /content/gdrive/MyDrive/datasets/test.zip /content/test.zip
#!unzip test.zip -d /content/test_dataset > /dev/null

In [7]:
# !rm -rf /content/gdrive/MyDrive/datasets/checkpoints/project_1/
# !mkdir /content/gdrive/MyDrive/datasets/project_1/checkpoints/

In [ ]:
!mkdir /content/run
%cd /content/run
!tar -xzvf /content/gdrive/MyDrive/datasets/project_1/checkpoints/last.tar.gz

In [ ]:
%cd /content/

from ultralytics import YOLO
import os
import shutil
from pathlib import Path

checkpoint_dir = f"/content/gdrive/MyDrive/datasets/project_1/checkpoints"
last_filename = "last.tar.gz"
last_gz_path = "{checkpoint_dir}/{last_filename}"

def on_fit_epoch_end(trainer):
    run_dir = Path("/content/") / trainer.save_dir.resolve()

    !tar -czvf last.tar.gz -C {run_dir} . > /dev/null

    if trainer.epoch > 0 and os.path.exists(last_gz_path):
      !mv "{last_gz_path}" "{checkpoint_dir}/epoch{trainer.epoch - 1}.tar.gz"

    !cp {last_filename} "{checkpoint_dir}"
    !rm {last_filename}


is_resume = False
model = "/content/runs/detect/scratch-detector/weights/last.pt"

if os.path.exists(model):
  is_resume = True
  print("Resume training")
else:
  model = 'yolo11m.pt'
  print("Start training")

model = YOLO(model)
model.add_callback('on_fit_epoch_end', on_fit_epoch_end)

results = model.train(
    data=f'/content/merged/data.yaml',  # Путь к YAML-файлу из Roboflow
    epochs=200,
    imgsz=640,
    batch=16,
    name="scratch-detector",
    device=0,  # Использование GPU; для CPU установите device='cpu',
    val=True,
    save=True,
    save_dir="/content/runs/",
    resume=is_resume,
    lr0=0.005,
    lrf=0.1,
    cos_lr=True,
    warmup_epochs=4,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    patience=30
)

/content
Start training
Ultralytics 8.3.155 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/merged/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=scratch-detector, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=30, perspective=0.0, plots

train: Scanning /content/merged/train/labels.cache... 22692 images, 93 backgrounds, 0 corrupt: 100%|██████████| 22692/22692 [00:00<?, ?it/s]

train: /content/merged/train/images/10231280-honda-honda_wr-v_i-vtec_vx-2020-minor_scratch-dents-kyzoprhnsn6t7wzygbbsew_jpeg.rf.727adb4bddc20c77b94e2b6e341c660c.jpg: 1 duplicate labels removed
train: /content/merged/train/images/10273265-mahindra-mahindra_tuv300_2015-2020_t10-2018-minor_scratch-dents-8qeegk1otmwprqb2qodwta_jpeg.rf.4ce5a9f8fc822aa8911fa89e34efcb38.jpg: 1 duplicate labels removed
train: /content/merged/train/images/8368960-hyundai-hyundai_elite_i20_2014-2018_asta_12-2014-minor_scratch-dents-a8mnu2soqlweeejh-2bsgcla_jpeg.rf.482e12f708419970542a1bd2367a7422.jpg: 1 duplicate labels removed
train: /content/merged/train/images/8717563-hyundai-hyundai_grand_i10_nios_asta_amt_12_kappa_vtvt-2021-minor_scratch-dents-og5xnheuqvsdx4qu3sasta_jpeg.rf.a0b6741977d96ee551f56c006069af7a.jpg: 1 duplicate labels removed
train: /content/merged/train/images/9271257-renault-renault_scala_2012-2018_rxl-2016-minor_scratch-dents-lunybi9brumj9nfmzzq_ta_jpeg.rf.45f2847db726cbf65174b53bfbe341ce.jpg

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 741.1±903.3 MB/s, size: 61.8 KB)


val: Scanning /content/merged/valid/labels.cache... 4862 images, 21 backgrounds, 0 corrupt: 100%|██████████| 4862/4862 [00:00<?, ?it/s]


Plotting labels to runs/detect/scratch-detector/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.005' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/scratch-detector
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/200      14.4G      1.565      2.807      1.711         14        640: 100%|██████████| 1419/1419 [14:16<00:00,  1.66it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 152/152 [01:20<00:00,  1.89it/s]


                   all       4862       9301       0.51      0.462      0.454      0.323

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      2/200      9.13G      1.514       2.23      1.631         64        640:  36%|███▌      | 510/1419 [05:02<08:28,  1.79it/s]

In [8]:
#!rm -rf /content/content
# !rm -rf /content/merged/
!rm -rf /content/runs/
#!rm -rf /content/gdrive/MyDrive/models/checkpoints/test/
#!mkdir -P /content/gdrive/MyDrive/models/checkpoints/test/
#!ls /content/gdrive/MyDrive/models/checkpoints/test/
